In [ ]:
# Run this cell to download dependencies
%pip install lingo_api==19.0.6
%pip install pandas
%pip install plotly
%pip install geopy

In [7]:
import lingo_api as lingo
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from geopy import distance

In [54]:
# TSP Callback
# state = 1: The LINGO model has just added new subtours to the cuts
# state = 2: The model has been solved
# state = 0: Working on solving the submodel or finding new cuts
#
#
def cbSolver(pEnv, nReserved, uData):
    state   = np.array([-1.0], dtype=np.double)
    errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["Model_State"], state)
    
    if state[0] == 1:

        newTours   = np.array([-1.0], dtype=np.double)
        totalTours = np.array([-1.0], dtype=np.double)
        kurTime = np.array([-1.0], dtype=np.double)
        
        errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["New_Tours"], newTours)
        errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["Total_Tours"], totalTours)
        errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["Time"], kurTime)
        
        print(f"Total Cuts: {int(totalTours[0]):-5} \t New Cuts: {int(newTours[0]):-5} \t Time Used: {kurTime[0]:-5.4f}")
    
    elif state[0] == 2 and uData["Solve_Printed"]==False:
        uData["Solve_Printed"] = True
          
        solved     = np.array([-1.0], dtype=np.double)
        newTours   = np.array([-1.0], dtype=np.double)
        totalTours = np.array([-1.0], dtype=np.double)
        kurTime    = np.array([-1.0], dtype=np.double)
        
        errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["SOLVED"], solved)
        errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["New_Tours"], newTours)
        errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["Total_Tours"], totalTours)
        errorcode = lingo.pyLSgetCallbackVarPrimalLng(pEnv, uData["Time"], kurTime)

        if solved[0] == 0:
            printStr = f"TSP Solved in {kurTime[0]:.4f} seconds"
        else:
            printStr = f"TSP was unable to be solved before running out of cuts"
        print("\n=======================================================================================")
        print(printStr)

    else:
        pass
    return 0

def cbError(pEnv, uData, nErrorCode, errorText):
    # A exception will be displayed if this callback is called
    raise(nErrorCode, errorText)

## LINGO - Python TSP

This TSP model is implemented using Dantzig, Fulkerson, Johnson (DFJ) subtour elimination constraints with the maximum number of cuts each iteration controlled by the user. The data for this model comes from US_Tour.csv, however the data can be swaped out with another data set with the columns; city, latitude, longitude. The LINGO model is included in the same directory as this jupyter notebook.

There are seven LINGO pointers that this model uses to send and recive data that must be set/allocated in Python.

- **Pointer1**: *MAXTOUR* a model parameter that sets the maximum number of subtours that can be added to the model.
- **Pointer2**: *MAXADD* a model parameter that sets the maximum number of subtours that can be added at each iteration.
- **Pointer3**: *CITY* a model set passed as a NumPy array of city names that will be visited on the tour.
- **Pointer4**: *LATI* a model parameter passed as a NumPy array of latitudes of each city on the tour.
- **Pointer5**: *LNGT* a model parameter passed as a NumPy array of longitudes of each city on the tour.
- **Pointer6**: *Y* a model variabel matrix passed as a NumPy array of size numberOfCities*numberOfCities where $Y_{i,j} = 1$ if path form city $i$ to city $j$ is used $0$ otherwise.
- **Pointer7**: *DIST* a model parameter for of distance between node $i$ and $j$. To allocate memmory for this return data create an empty NumPy array of size numberOfCities*numberOfCities.
- **Pointer8**: *SOLVED* a model variable that is is $0$ if the TSP was solved $1$ otherwise.

In [57]:
# csv data
df = pd.read_csv("US_Tour.csv")
# model 
lngFile = "tspLoop.lng"

# data for pointres
CITY = df["city"].values
LATI = df["lati"].values  
LNGT = df["lngt"].values

        
MAXTOUR = 50
MAXADD  = 10
numCities = len(CITY)

Y = np.empty(numCities*numCities)
DIST = np.empty(numCities*numCities)
SOLVED = np.array([-1.0])

# Callback Data
uData =  {"Prefix":"Lingo API",
          "Model_State":np.array(["STATE"],dtype="|S102"),
          "New_Tours":np.array(["ADDED"],dtype="|S102"),
          "Total_Tours":np.array(["TOTALTOURS"],dtype="|S102"),
          "Time":np.array(["KURTIME"],dtype="|S102"),
          "SOLVED":np.array(["SOLVED"],dtype="|S102"),
          "Solve_Printed":False,
}


In [58]:
# Create a model object
model = lingo.Model(lngFile)

# set all pointers in the order that they appear in tspLoop.lng
model.set_pointer("Pointer1",MAXTOUR,lingo.PARAM)
model.set_pointer("Pointer2",MAXADD,lingo.PARAM)
model.set_pointer("Pointer3",CITY,lingo.SET)
model.set_pointer("Pointer4",LATI,lingo.PARAM)
model.set_pointer("Pointer5",LNGT,lingo.PARAM)
model.set_pointer("Pointer6",Y,lingo.VAR)
model.set_pointer("Pointer7",DIST,lingo.PARAM)
model.set_pointer("Pointer8",SOLVED,lingo.VAR)

# set all callback functions and user data
model.set_cbSolver(cbSolver)
model.set_uData(uData)

# solve the model
lingo.solve(model)

# Fix Data
Y = np.reshape(Y,(numCities,numCities))
DIST = np.reshape(DIST,(numCities,numCities))

Total Cuts:    10 	 New Cuts:    10 	 Time Used: 0.0960
Total Cuts:    20 	 New Cuts:    10 	 Time Used: 0.1760
Total Cuts:    30 	 New Cuts:    10 	 Time Used: 0.2470
Total Cuts:    39 	 New Cuts:     9 	 Time Used: 0.3880
Total Cuts:    43 	 New Cuts:     4 	 Time Used: 0.4660
Total Cuts:    44 	 New Cuts:     1 	 Time Used: 0.5830

TSP Solved in 0.5830 seconds


In [59]:
# Orginaize data to be ploted
j_map = np.zeros(numCities, dtype=np.int32)


pos = 0
for ij, val in np.ndenumerate(Y):
    if val == 1:
        i = ij[0]
        j = ij[1]
        j_map[pos] = j
        pos += 1 
        
entry_pos = 0
lon_sorted = np.zeros(numCities)
lat_sorted = np.zeros(numCities)
city_sorted = np.empty(numCities, dtype=object)
sorted_indx = np.zeros(numCities,dtype=int)

lon_sorted[entry_pos] = LNGT[0]
lat_sorted[entry_pos] = LATI[0]
city_sorted[0] = CITY[0]
sorted_indx[0] = 0


for i in range(1,len(j_map)-1):
    lon_sorted[i] = LNGT[j_map[entry_pos]]
    lat_sorted[i] = LATI[j_map[entry_pos]]
    city_sorted[i] = CITY[j_map[entry_pos]]
    sorted_indx[i] = j_map[entry_pos]
    entry_pos = j_map[entry_pos]
    
lon_sorted[-1] = LNGT[0]
lat_sorted[-1] = LATI[0]
city_sorted[-1] = CITY[0]
sorted_indx[-1] = 0

In [60]:
SOLVED

array([0.])

In [61]:
fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = lon_sorted,
    lat = lat_sorted,
    text = city_sorted,
    marker = {'size': 10, 'color' : 'red'}))


fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': -98.35, 'lat': 39.50},
        'style': "stamen-terrain",
        'zoom': 3})


fig.show()

In [53]:
header = ['City', '', 'total Dist']
ijdist = np.empty((len(city_sorted)))
accDist = np.empty((len(city_sorted)))

ijdist[0] = 0
accDist[0] = 0

for k in range(1, len(sorted_indx)):
    i = sorted_indx[k-1]
    j = sorted_indx[k]
    ijdist[k] = DIST[i,j]
    accDist[k] = accDist[k-1] + DIST[i,j]


tspDF = pd.DataFrame({'City':city_sorted, 'dist':ijdist, 'total Dist':accDist})
tspDF

,City,dist,total Dist
0,Bellingham,0.000000,0.000000
1,Seattle,132.798715,132.798715
2,Portland,224.046870,356.845584
3,Fresno,1010.769660,1367.615244
4,Los_Angeles,330.516511,1698.131755
5,LongBeach,27.233474,1725.365229
6,SanDiego,151.487074,1876.852303
7,LasVegas,422.195674,2299.047977
8,Phoenix,413.264829,2712.312806
9,Tucson,176.170847,2888.483653
